In [7]:
#plotting imports
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# ignore warnings
import warnings
warnings.filterwarnings("ignore")


#modeling imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#pipeline modules
from acquire import get_titanic_data
from prepare import prep_titanic

In [8]:

df=prep_titanic(get_titanic_data())
df.dropna(inplace=True)
df.isnull().sum()

NameError: name 'titanic_df' is not defined

In [ ]:
X=df[['pclass','age','fare','sibsp','parch']]
y=df[['survived']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.30,random_state=123)
X_train.head()

In [ ]:
logit=LogisticRegression(C=1,class_weight={1:2},random_state=123,solver='saga')
logit.fit(X_train,y_train)

In [ ]:
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

In [ ]:
y_pred = logit.predict(X_train)
y_pred_proba=logit.predict_proba(X_train)

y_pred_proba

In [ ]:
print('Accuracy of Logistic regression on training set: {:.2f}'.format(logit.score(X_train,y_train)))

In [ ]:
print(confusion_matrix(y_train,y_pred))

In [ ]:
c_mat=confusion_matrix(y_train,y_pred)

In [ ]:
false_positive=c_mat[0,1]
true_negative=c_mat[0,0]
false_negative=c_mat[1,0]
true_positive=c_mat[1,1]

In [ ]:
#compare this to what was produced from score method
accuracy=((true_negative+true_positive)/len(y_train))
accuracy

In [ ]:
#1-accuracy
classification_error=((false_positive+false_negative)/len(y_train))
classification_error

In [ ]:
#also called true positive rate
recall=true_positive/(true_positive+false_negative)
recall

In [ ]:
precision=true_positive/(true_positive+false_positive)
precision

In [ ]:
f1_score=(recall+precision)/2
f1_score

In [ ]:
false_positive_rate=false_positive/(false_positive+true_negative)
false_positive_rate

In [ ]:
cr=(classification_report(y_train,y_pred,output_dict=True))
#print(cr["0"])
#print(cr["1"])

In [ ]:
print('Accuracy of Logistic Regression classifier on test set: {:.2f}'.format(logit.score(X_test,y_test)))

In [ ]:
y_pred_proba=[i[1] for i in y_pred_proba]

In [ ]:
fig=plt.figure()
ax=fig.add_subplot(111)
ax.scatter(y_pred_proba,y_pred)
plt.show()

Let's us a decision tree to solve a classification problem

In [ ]:
import pandas as pd
from pydataset import data
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from pydataset import data

In [ ]:
df_iris=data('iris')
df_iris.head()

In [ ]:
df_iris.columns=[col.lower().replace('.','_') for col in df_iris]

In [ ]:
df_iris.head()
X=df_iris.drop(['species'],axis=1)
y=df_iris[['species']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.30,random_state=123)

In [ ]:
clf=DecisionTreeClassifier(criterion='entropy',max_depth=3,random_state=123)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
y_pred=clf.predict(X_train)
y_pred_proba=clf.predict_proba(X_train)
y_pred_proba
#y_pred

In [ ]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf.score(X_train,y_train)))

In [ ]:
confusion_matrix(y_train,y_pred)

In [ ]:
#Making the confusion matrix "pretty"
labels=sorted(y_train.species.unique())
pretty_cr=pd.DataFrame(confusion_matrix(y_train,y_pred),index=labels,columns=labels)